In [1]:
import cv2
import face_recognition
import numpy as np
from PIL import Image, ImageDraw

/home/juan-jose/tf-gpu1/lib/python3.12/site-packages/face_recognition_models/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
face1Image = face_recognition.load_image_file("content/mari.jpeg")
face2Image = face_recognition.load_image_file("content/juan.jpeg")
face3Image = face_recognition.load_image_file("content/pexels-mateus.jpg")
videosCapture = cv2.VideoCapture(0)

In [3]:

face1Encondings = face_recognition.face_encodings(face1Image)[0]#importante para que nos retorne el vector y no la matriz
face2Encondings = face_recognition.face_encodings(face2Image)[0]
face3Encondings = face_recognition.face_encodings(face3Image)[0]
known_face_encodings = [face1Encondings,face2Encondings,face3Encondings]


In [4]:
known_face_names = ["mari", 
              "juan",
              "mateus"]


In [ ]:
import face_recognition
import cv2
import numpy as np

video_capture = cv2.VideoCapture(0)



process_this_frame = True
while True:
    # Grab a single frame of video
    ret, frame = videosCapture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)#ojo, no volver a usar [:,:,::-1] eso fue lo que me estaba jodiendo 
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            #matches = face_recognition.compare_faces(known_face_encodings, face_encoding) // esto lo que hace es comparar las ditancias, y arroja una lista de verdaderos
            #o falsos segun la distancia, se le puede poner la tolerancia, siento 0.4 muy estricto y 0.8 poco estricto, basicamente es la forma simplificada de el codigo
            #de abajo
            name = "Unknown"

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            #if matches[best_match_index]:
              #  name = known_face_names[best_match_index]
            if face_distances[best_match_index]<0.6:
                name = known_face_names[best_match_index]
                face_names.append(name)

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

[ WARN:0@250.570] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@250.570] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range
